# Behavior Elements

## 1. Feeding
Feeding from the food source (yellow/red)

## 2. Unloading
Non-receiving interactions

## 3. Receiving
Receiving interactions (more than 'min_receive' ul)

## 4. Resting
Not feeding, not interacting, not moving (less than 'min_move' pixels/ 'dt' frames)

## 5. Moving
Not feeding, not interacting, moving more than 'min_move' pixels/ 'dt' frames

In [78]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData
from os import sep as sep
import AnalysisFunctions as afuns
from functools import reduce
import os

In [79]:
root_path = r'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')

In [80]:
def get_bdata_filename(exp_path):
    bdata_filename = [filename for filename in os.listdir(exp_path+sep+r'with food\blob analysis normalized by white paper') 
                        if filename.startswith(r'bdata')][0]
    return bdata_filename

# Load experiment data

In [190]:
exp_num = 20
exp_idx, PC_ratios_dict, exp_path = afuns.get_exp_data(exp_num, exp_details, root_path)
bdata_filename = get_bdata_filename(exp_path)

bdata = pd.read_csv(exp_path + sep + r'with food\blob analysis normalized by white paper' + sep + bdata_filename, index_col=0)
fdata = pd.read_csv(exp_path + sep + r'forager_table_with_feeding_sizes_ul_transparency_corrected.csv', index_col=[0])
tdata = pd.read_csv(exp_path + sep + r'clean_trophallaxis_table_transparency_corrected.csv')
crops = pd.read_csv(exp_path + sep + r'clean_crops_transparency_corrected.csv', header=[0,1], index_col=[0]).swaplevel(axis=1)
ants = pd.read_csv(exp_path + sep + r'ants_list.csv')
conversion_factors = pd.read_csv(exp_path + sep + r'conversion_factors_by_weight_and_feeding_sum.csv')
transparency = pd.read_csv(exp_path + sep + r'transparency_table.csv')

# Parameters

In [202]:
min_volume = 1  # ul
dt = 10  # frames
min_move = 50  # pixels/dt frames

smoothing_window = 70 # frames

# Functions

In [94]:
def plot_event(ax,start_frame, end_frame, color, alpha):
    ax.axvspan(start_frame, end_frame, facecolor=color, alpha=alpha, zorder=-100)

In [84]:
def get_ant_data(ant, bdata, crops, tdata, transparency, dt=10, fdata=False, conversion_factors=False):
    ant_data={}
    
    ant_data['ant'] = ant
    
    raw_x = bdata['a'+str(ant)+'-x']
    raw_y = bdata['a'+str(ant)+'-y']
    ant_data['x']=raw_x.interpolate(method='linear',limit_area='inside')
    ant_data['y']=raw_y.interpolate(method='linear',limit_area='inside')
    
    ant_data['v']=calc_vel(ant_data['x'],ant_data['y'],dt)
    
    ant_data['transp'] = transparency.transparency[transparency['ant']==ant].iloc[0]
    
    raw_crop = bdata['a'+str(ant)+'-crop_intensity']/ant_data['transp']/ant_data['transp']
    
    crop_df_raw = pd.DataFrame({'frame':raw_crop.index, 'raw_crop':raw_crop.values, 'acquisition':bdata['acquisition'].values})
    crop_df_clean = pd.DataFrame({'clean_red':crops[('red',str(ant))], 'clean_yellow':crops[('yellow',str(ant))]})
    crop_df = crop_df_raw.merge(crop_df_clean,left_index=True, right_index=True)
    
    if isinstance(conversion_factors, pd.DataFrame):
        crop_df['raw_crop'][crop_df['acquisition']=='BLGF']=crop_df['raw_crop'][crop_df['acquisition']=='BLGF']/conversion_factors['yellow'][0]
        crop_df['raw_crop'][crop_df['acquisition']=='GLRF']=crop_df['raw_crop'][crop_df['acquisition']=='GLRF']/conversion_factors['red'][0]
        crop_df['clean_red']=crop_df['clean_red']/conversion_factors['red'][0]
        crop_df['clean_yellow']=crop_df['clean_yellow']/conversion_factors['yellow'][0]
        
    ant_data['crop_df']=crop_df
    
    ant_data['interactions']=tdata[(tdata['giver']==ant) | (tdata['receiver']==ant)]
    
    if isinstance(fdata, pd.DataFrame):
        ant_data['feedings'] = fdata[fdata.ant_id==ant]
    
    return ant_data


def calc_vel(x,y,dt):
    diff_x = x.diff(periods=dt)
    diff_y = y.diff(periods=dt)
    v = np.sqrt(diff_x**2 + diff_y**2)/dt
    return v

In [85]:
def get_intervals_between_events(ant_data):
    interactions = pd.DataFrame()
    interactions['start_frame'] = ant_data['interactions']['start_frame']
    interactions['end_frame'] = ant_data['interactions']['end_frame']
    interactions['event_type'] = 'interaction'

    feedings = pd.DataFrame()
    feedings['start_frame'] = ant_data['feedings']['feeding_start']
    feedings['end_frame'] = ant_data['feedings']['feeding_end']
    feedings['event_type'] = 'feeding'

    all_events = pd.concat([interactions, feedings]).sort_values(by='start_frame')
    all_events_df = all_events.copy()

    interval_starts=[0]
    interval_ends=[]
    while len(all_events)>0:

        next_event = all_events.iloc[0]
        next_event_start = next_event['start_frame']
        next_events = all_events[all_events['start_frame']<=next_event['end_frame']]  # to deal with overlapping events
        next_event_end = max(next_events['end_frame'])  # if next events are overlapping - take the end of the one that ends last
        
        interval_ends.append(next_event_start)
        interval_starts.append(next_event_end+1)
        
        all_events = update_remaining_events(interval_starts[-1], all_events)

    interval_ends.append(bdata['frame'].iloc[-1])
    
    return interval_starts, interval_ends, all_events_df
    

def update_remaining_events(frame, events):
    events=events[events['start_frame']>=frame]
    return events

In [86]:
# Classify interactions to unloading, receiving and small
def classify_interactions(behavior_df, ant_data, min_volume):
    giving_df = ant_data['interactions'][ant_data['interactions']['giver']==ant]
    unloading = giving_df[['start_frame','end_frame']][(giving_df['transferred_red']>=min_volume) | (giving_df['transferred_yellow']>=min_volume)]
    small_giving = giving_df[['start_frame','end_frame']][(giving_df['transferred_red']<min_volume) | (giving_df['transferred_yellow']<min_volume)]
    
    receiving_df = ant_data['interactions'][ant_data['interactions']['receiver']==ant]
    receiving = receiving_df[['start_frame','end_frame']][(receiving_df['transferred_red']>=min_volume) | (receiving_df['transferred_yellow']>=min_volume)]
    small_receiving = receiving_df[['start_frame','end_frame']][(receiving_df['transferred_red']<min_volume) | (receiving_df['transferred_yellow']<min_volume)]
    
    small_interactions = pd.concat([small_giving, small_receiving])
    
    for df, behavior in zip([unloading, receiving, small_interactions], ['non-receiving_interaction','receiving_interaction','non-receiving_interaction']):
        df['behavior'] = behavior
    
    behavior_df = pd.concat([behavior_df, unloading, receiving, small_interactions])
    behavior_df = behavior_df.sort_values(by='start_frame').reset_index(drop=True)
    
    return behavior_df

In [87]:
# insert feedings to behavior_df
def insert_feedings(behavior_df, ant_data):
    feedings_df = ant_data['feedings'][['feeding_start','feeding_end','food_source']]
    feedings_df=feedings_df.rename(columns={'feeding_start':'start_frame', 'feeding_end':'end_frame','food_source':'behavior'})
    feedings_df['behavior']=feedings_df['behavior'].apply(lambda x: x + '_feeding')
    
    behavior_df = pd.concat([behavior_df, feedings_df])
    behavior_df = behavior_df.sort_values(by='start_frame').reset_index(drop=True)
    
    return behavior_df

In [88]:
def merge_consecutive_events(behavior_df, window_size):
    behavior_df['next_behavior']=behavior_df['behavior'].shift(-1)
    behavior_df['next_start']=behavior_df['start_frame'].shift(-1)
    behavior_df['next_end']=behavior_df['end_frame'].shift(-1)
    
    group_labels=[]
    counter = 1
    chunk_ends = []
    in_chunk = False
    for idx, event in behavior_df.iterrows():
        if not to_merge(event, window_size):
            group_labels.append(0)
            counter += 1
            if in_chunk:
                chunk_ends.append(idx)
            in_chunk=False
        else:
            group_labels.append(counter)
            in_chunk=True
    
    behavior_df['group_labels']=group_labels
    
    groups = np.unique(group_labels)
    groups = groups[groups>0]
    
    merged_events=[]
    for g in groups:
        chunk = behavior_df[behavior_df['group_labels']==g]
        merged_event = chunk.iloc[0][['start_frame','end_frame','behavior']]
        merged_event['end_frame']=chunk.iloc[-1]['next_end']
        merged_events.append(merged_event)
    
    merged_df = pd.concat(merged_events,axis=1).transpose()
    solitary_events = behavior_df[behavior_df['group_labels']==0]
    solitary_events=solitary_events.drop(index=chunk_ends)
    behavior_df = pd.concat([merged_df,solitary_events[['start_frame','end_frame','behavior']]]).sort_values(by='start_frame')
    behavior_df=behavior_df.reset_index(drop=True)
    
    return behavior_df

def to_merge(event, window_size):
        m = (event['behavior']==event['next_behavior']) & ((event['next_start']-event['end_frame'])<=window_size)
        return m

In [89]:
def is_unloading(merged_event, crop_df, min_vol):
    red_change = crop_df['clean_red'][merged_event['start_frame']]-crop_df['clean_red'][merged_event['end_frame']]
    yellow_change = crop_df['clean_yellow'][merged_event['start_frame']]-crop_df['clean_yellow'][merged_event['end_frame']]
    res = (red_change>min_vol) | (yellow_change>min_vol)
    return res

# Single ant

In [149]:
ant = 148
ant_data = get_ant_data(ant, bdata, crops, tdata, transparency, dt=dt, fdata=fdata, conversion_factors=conversion_factors)

C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [150]:
behavior_df = pd.DataFrame()
behavior_df = classify_interactions(behavior_df, ant_data, min_volume)
behavior_df = insert_feedings(behavior_df, ant_data)
merged_df = merge_consecutive_events(behavior_df, 70)


# Combined Elements

## Unloading
Merged non-receiving interactions that result in a decrease of more than min_vol

In [151]:
unloading_periods = merged_df.apply(is_unloading,axis=1,args=(ant_data['crop_df'], min_volume))
merged_df['behavior'][unloading_periods]='unloading'
merged_df['next_behavior']=merged_df['behavior'].shift(-1)
merged_df['next_start']=merged_df['start_frame'].shift(-1)
merged_df['next_end']=merged_df['end_frame'].shift(-1)

## Feeding bout
Feeding + Unloading

In [152]:
combined_behaviors=[]
rows_to_drop=[]
for idx, behavior in merged_df.iterrows():
    combined_behavior=behavior[['start_frame','end_frame','behavior']]
    if behavior['behavior'].endswith('feeding') & (behavior['next_behavior']=='unloading') & ((behavior['next_start']-behavior['end_frame'])<smoothing_window):
        combined_behavior['behavior']=behavior['behavior'].replace('feeding','bout')
        combined_behavior['end_frame'] = behavior['next_end']
        rows_to_drop.append(idx+1)
    combined_behaviors.append(combined_behavior)

combined_df=pd.concat(combined_behaviors,axis=1).transpose()
combined_df=combined_df.drop(index=rows_to_drop)

In [153]:
colors_dict = {'yellow_feeding':'y','red_feeding':'red',
               'non-receiving_interaction':'0.2', 'receiving_interaction':'g', 'unloading':'b',
              'red_bout':'r', 'yellow_bout':'y'}
alpha_dict={'red_bout':0.2,'yellow_bout':0.2,'non-receiving_interaction':0.2,'receiving_interaction':0.2, 'unloading':0.2,
            'red_feeding':0.6,'yellow_feeding':0.6}

In [154]:
%matplotlib qt
fig,ax = plt.subplots(3,1,figsize=[18, 12])
for idx, event in behavior_df.iterrows():
    plot_event(ax[0] ,event['start_frame'], event['end_frame'],colors_dict[event['behavior']], 0.2)
for idx, event in merged_df.iterrows():
    plot_event(ax[1] ,event['start_frame'], event['end_frame'],colors_dict[event['behavior']], 0.2)
for idx, event in combined_df.iterrows():
    plot_event(ax[2] ,event['start_frame'], event['end_frame'],colors_dict[event['behavior']], alpha_dict[event['behavior']])

In [131]:
# filter merged unloading periods by change in crop
# crop must decrease by more than min_vol

In [55]:
fdata.head()

,ant_id,crop_after,crop_before,feeding_end,feeding_size_intensity,feeding_start,first_interaction_after_partner,first_interaction_after_start,food_source,last_interaction_before_end,last_interaction_before_partner,feeding_size_ul
0,76,210171.501709,0.000000,58,210171.501709,4,NaN,NaN,yellow,NaN,NaN,4.513998
1,148,321006.945312,0.000000,128,321006.945312,48,NaN,NaN,yellow,NaN,NaN,5.128422
2,289,493876.082275,0.000000,304,493876.082275,133,NaN,NaN,yellow,NaN,NaN,6.396295
3,392,441520.468750,120736.955469,206,320783.513281,159,NaN,NaN,yellow,NaN,NaN,4.920353
4,289,489070.875000,333051.796875,395,156019.078125,371,NaN,NaN,yellow,NaN,NaN,2.020637


30

# All foragers

In [194]:
foragers = list(ants['ant_id'][ants['is_forager']])

foragers_df = pd.DataFrame()
num_feedings=[]
num_yellow_feedings=[]
# sort foragers by number of feedings and then by number of yellow feedings
for ant in foragers:
    ant_fdata=fdata[fdata['ant_id']==ant]
    num_feedings.append(len(ant_fdata))
    num_yellow_feedings.append(sum(ant_fdata['food_source']=='yellow'))

foragers_df['ant']=foragers
foragers_df['num_feedings']=num_feedings
foragers_df['num_yellow_feedings']=num_yellow_feedings

foragers_df.sort_values(by=['num_feedings','num_yellow_feedings'],inplace=True,ascending=False)
sorted_foragers=foragers_df['ant']
sorted_foragers

6    1235
5    1124
0    1299
2     386
4      61
1     236
7     150
3     424
8     673
Name: ant, dtype: int64

In [195]:
fig,ax = plt.subplots(len(foragers),1,figsize=[12, 12],sharex=True)

counter=0                      
for ant in sorted_foragers:
    
    ant_data = get_ant_data(ant, bdata, crops, tdata, transparency, dt=dt, fdata=fdata, conversion_factors=conversion_factors)
    
    behavior_df = pd.DataFrame()
    behavior_df = classify_interactions(behavior_df, ant_data, min_volume)
    behavior_df = insert_feedings(behavior_df, ant_data)
    merged_df = merge_consecutive_events(behavior_df, 70)
    
    unloading_periods = merged_df.apply(is_unloading,axis=1,args=(ant_data['crop_df'], min_volume))
    merged_df['behavior'][unloading_periods]='unloading'
    merged_df['next_behavior']=merged_df['behavior'].shift(-1)
    merged_df['next_start']=merged_df['start_frame'].shift(-1)
    merged_df['next_end']=merged_df['end_frame'].shift(-1)
    
    combined_behaviors=[]
    rows_to_drop=[]
    for idx, behavior in merged_df.iterrows():
        combined_behavior=behavior[['start_frame','end_frame','behavior']]
        if behavior['behavior'].endswith('feeding') & (behavior['next_behavior']=='unloading') & ((behavior['next_start']-behavior['end_frame'])<smoothing_window):
            combined_behavior['behavior']=behavior['behavior'].replace('feeding','bout')
            combined_behavior['end_frame'] = behavior['next_end']
            rows_to_drop.append(idx+1)
        combined_behaviors.append(combined_behavior)

    combined_df=pd.concat(combined_behaviors,axis=1).transpose()
    combined_df=combined_df.drop(index=rows_to_drop)
                      
    for idx, event in combined_df.iterrows():
        plot_event(ax[counter] ,event['start_frame'], event['end_frame'],colors_dict[event['behavior']], alpha_dict[event['behavior']])
    
    ax[counter].set_yticks([])
    ax[counter].set_xlim([0, bdata['frame'].iloc[-1]])
    ax[counter].set_ylabel(ant,rotation=0)
    ax[counter].yaxis.set_label_coords(-0.025,0.35)
                      
    counter+=1
    
fig.suptitle(f"Exp. {exp_num}")

C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Text(0.5, 0.98, 'Exp. 20')

## Non-event intervals

In [196]:
interval_starts, interval_ends, all_events = get_intervals_between_events(ant_data)

In [215]:
%matplotlib qt
fig,ax = plt.subplots(figsize=[18, 4])
for idx, event in ant_data['feedings'].iterrows():
    plot_event(ax,event['feeding_start'], event['feeding_end'],event['food_source'], 0.3)
for idx, event in ant_data['interactions'].iterrows():
            plot_event(ax,event['start_frame'], event['end_frame'],'0.5', 0.2)
for t0,t1 in zip(interval_starts, interval_ends):
    plot_event(ax,t0,t1,'blue',0.2)
plt.plot(ant_data['v']/ant_data['v'].max(skipna=True))
plt.plot(ant_data['v']>min_move,'ob')
plt.plot(smoothed_moving,'r')
plt.plot(np.ceil(smoothed_moving),'ok')

In [214]:
moving = ant_data['v']>40#min_move
smoothed_moving = np.ceil(moving.rolling(70, min_periods=1, center=True).mean()) #smoothing_window

for idx,event in all_events.iterrows():
    smoothed_moving.iloc[int(event['start_frame']):int(event['end_frame'])]=0

fig = plt.figure(figsize=[18, 4])
plt.plot(ant_data['v']/ant_data['v'].max(skipna=True))
plt.plot(ant_data['v']>min_move,'ob')
plt.plot(smoothed_moving,'or')
plt.plot(np.ceil(smoothed_moving),'ok')

In [211]:
def get_moving_and_resting_in_interval(interval_start, interval_end, raw_moving, smoothing_window):
    raw_moving = raw_moving.iloc[int(interval_start):int(interval_end)]
    
    changes = np.diff(np.multiply(raw_moving,1))
    resting_starts=np.argwhere(changes==-1).flatten()
    moving_starts=np.argwhere(changes==1).flatten()
    initial_state = raw_moving[0]
    
    
    # if no changes, the whole interval is the initial state
    if ~np.any(changes):
        if initial_state:
            moving_starts.append(interval_start)
            moving_ends.append(interval_end)
        else:
            resting_starts.append(interval_start)
            resting_ends.append(interval_end)
    
    # if there are changes
    else:
        # treat edges
        if resting_starts[0]>moving_starts[0]:
            resting_starts=np.insert(resting_starts,0,interval_start)
        if resting_starts[-1]>moving_starts[-1]:
            moving_starts=np.append(moving_starts,interval_end)
            
        # remove resting shorter than smoothin
            
    
    
    return moving_starts, moving_ends, resting_starts, resting_ends
            
    

50

In [259]:
def get_moving_and_resting_intervals(raw_moving, smoothing_window, interval_starts, interval_ends):
    
    # get "resting start"s and "moving start"s
    changes = np.diff(np.multiply(raw_moving,1))
    resting_starts=np.argwhere(changes==-1).flatten()
    moving_starts=np.argwhere(changes==1).flatten()
    if ~raw_moving.iloc[0]:
        resting_starts=np.insert(resting_starts,0,0)
    
    if len(resting_starts)>len(moving_starts):
        moving_starts = np.append(moving_starts,raw_moving.index[-1])
    
    if len(resting_starts)<len(moving_starts):
        moving_starts = moving_starts[1:]
    
    # for every "resting start", check if next "moving start" is within smoothing_window.
    # if yes: remove this resting start and moving start
    idxs_to_remove=[]
    counter=0
    for r_start, m_start in zip(resting_starts, moving_starts):
        if (m_start-r_start) < smoothing_window:
            idxs_to_remove.append(counter)
        counter+=1
    moving_starts=np.delete(moving_starts,idxs_to_remove)
    resting_starts=np.delete(resting_starts,idxs_to_remove)
    
    resting_ends=moving_starts-1
    
    if resting_starts[0]>0:
        moving_starts = np.insert(moving_starts,0,0)
        moving_ends=resting_starts-1
    else:
    
            
    
    return moving_starts, resting_starts

In [243]:
moving.index[-1]

10856

In [219]:
np.diff([True, True, False, False, False, True])

array([False,  True, False, False,  True])

In [221]:
[True, True, False, False, False, True]*1

[True, True, False, False, False, True]

In [223]:
np.multiply([True, True, False, False, False, True],1)

array([1, 1, 0, 0, 0, 1])

In [230]:
a = np.diff(np.multiply(moving,1))
a

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [234]:
a[76]

-1

In [236]:
j = np.argwhere(a==-1).flatten()
j

array([   39,    76,    90,   219,   328,   378,   398,   426,   439,
         610,   620,   657,   667,   685,   702,   704,   717,   728,
         736,   746,   758,   762,   785,   816,   820,   839,   864,
         872,  1078,  1116,  1127,  1206,  1221,  1239,  1242,  1442,
        1690,  1925,  1958,  2149,  2179,  2181,  2425,  2827,  2884,
        2909,  2915,  2918,  2935,  2959,  3504,  3510,  3538,  3559,
        3575,  3983,  4380,  4685,  4708,  4722,  4727,  4734,  4748,
        5190,  5786,  5812,  5897,  6611,  6622,  6641,  6657,  6815,
        6827,  7001,  7331,  7350,  7521,  7584,  7602,  7611,  7613,
        7842,  7852,  8174,  8217,  8237,  8551,  8565,  8579,  8590,
        8703,  8716,  8749,  9761,  9788,  9801,  9807,  9810,  9824,
       10824, 10847], dtype=int64)

In [239]:
np.insert(j, 0, 0)

array([    0,    39,    76,    90,   219,   328,   378,   398,   426,
         439,   610,   620,   657,   667,   685,   702,   704,   717,
         728,   736,   746,   758,   762,   785,   816,   820,   839,
         864,   872,  1078,  1116,  1127,  1206,  1221,  1239,  1242,
        1442,  1690,  1925,  1958,  2149,  2179,  2181,  2425,  2827,
        2884,  2909,  2915,  2918,  2935,  2959,  3504,  3510,  3538,
        3559,  3575,  3983,  4380,  4685,  4708,  4722,  4727,  4734,
        4748,  5190,  5786,  5812,  5897,  6611,  6622,  6641,  6657,
        6815,  6827,  7001,  7331,  7350,  7521,  7584,  7602,  7611,
        7613,  7842,  7852,  8174,  8217,  8237,  8551,  8565,  8579,
        8590,  8703,  8716,  8749,  9761,  9788,  9801,  9807,  9810,
        9824, 10824, 10847], dtype=int64)

In [240]:
j.insert(0,0)

AttributeError: 'numpy.ndarray' object has no attribute 'insert'

In [241]:
j.append(11000)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [242]:
np.append(j,11000)

array([   39,    76,    90,   219,   328,   378,   398,   426,   439,
         610,   620,   657,   667,   685,   702,   704,   717,   728,
         736,   746,   758,   762,   785,   816,   820,   839,   864,
         872,  1078,  1116,  1127,  1206,  1221,  1239,  1242,  1442,
        1690,  1925,  1958,  2149,  2179,  2181,  2425,  2827,  2884,
        2909,  2915,  2918,  2935,  2959,  3504,  3510,  3538,  3559,
        3575,  3983,  4380,  4685,  4708,  4722,  4727,  4734,  4748,
        5190,  5786,  5812,  5897,  6611,  6622,  6641,  6657,  6815,
        6827,  7001,  7331,  7350,  7521,  7584,  7602,  7611,  7613,
        7842,  7852,  8174,  8217,  8237,  8551,  8565,  8579,  8590,
        8703,  8716,  8749,  9761,  9788,  9801,  9807,  9810,  9824,
       10824, 10847, 11000], dtype=int64)

In [260]:
moving_starts, resting_starts = get_moving_and_resting_intervals(moving, smoothing_window, [])
moving_starts
   

array([   10,    41,    81,   210,   324,   352,   379,   416,   438,
         609,   619,   645,   663,   676,   697,   703,   706,   719,
         731,   738,   756,   760,   774,   794,   818,   828,   848,
         871,  1077,  1099,  1117,  1203,  1211,  1223,  1240,  1427,
        1643,  1917,  1949,  2136,  2154,  2180,  2418,  2817,  2861,
        2906,  2910,  2916,  2932,  2939,  3493,  3508,  3527,  3544,
        3566,  3978,  4372,  4666,  4690,  4720,  4724,  4733,  4737,
        5180,  5762,  5804,  5875,  6601,  6618,  6626,  6646,  6797,
        6817,  6981,  7330,  7339,  7511,  7561,  7587,  7609,  7612,
        7841,  7849,  8168,  8190,  8227,  8531,  8555,  8568,  8581,
        8697,  8704,  8737,  9755,  9777,  9792,  9806,  9809,  9815,
       10815, 10846, 10851], dtype=int64)

In [261]:
resting_starts

array([    0,    39,    76,    90,   219,   328,   378,   398,   426,
         439,   610,   620,   657,   667,   685,   702,   704,   717,
         728,   736,   746,   758,   762,   785,   816,   820,   839,
         864,   872,  1078,  1116,  1127,  1206,  1221,  1239,  1242,
        1442,  1690,  1925,  1958,  2149,  2179,  2181,  2425,  2827,
        2884,  2909,  2915,  2918,  2935,  2959,  3504,  3510,  3538,
        3559,  3575,  3983,  4380,  4685,  4708,  4722,  4727,  4734,
        4748,  5190,  5786,  5812,  5897,  6611,  6622,  6641,  6657,
        6815,  6827,  7001,  7331,  7350,  7521,  7584,  7602,  7611,
        7613,  7842,  7852,  8174,  8217,  8237,  8551,  8565,  8579,
        8590,  8703,  8716,  8749,  9761,  9788,  9801,  9807,  9810,
        9824, 10824, 10847], dtype=int64)

In [265]:
a=[1,2,3]
a.append(4)
a

[1, 2, 3, 4]

In [268]:
np.delete(resting_starts,[0,1])

array([   76,    90,   219,   328,   378,   398,   426,   439,   610,
         620,   657,   667,   685,   702,   704,   717,   728,   736,
         746,   758,   762,   785,   816,   820,   839,   864,   872,
        1078,  1116,  1127,  1206,  1221,  1239,  1242,  1442,  1690,
        1925,  1958,  2149,  2179,  2181,  2425,  2827,  2884,  2909,
        2915,  2918,  2935,  2959,  3504,  3510,  3538,  3559,  3575,
        3983,  4380,  4685,  4708,  4722,  4727,  4734,  4748,  5190,
        5786,  5812,  5897,  6611,  6622,  6641,  6657,  6815,  6827,
        7001,  7331,  7350,  7521,  7584,  7602,  7611,  7613,  7842,
        7852,  8174,  8217,  8237,  8551,  8565,  8579,  8590,  8703,
        8716,  8749,  9761,  9788,  9801,  9807,  9810,  9824, 10824,
       10847], dtype=int64)

In [269]:
a = [1,1,1]
np.diff(a)

array([0, 0])

In [270]:
np.any(np.diff(a))


False